In [1]:
# EXECUTE THIS BLOCK FIRST FOR INITIALISATION!

# dependencies
# ==========
# jupyter_server_mathjax
# ipywidget
# ipysheet
# sympy
# pandas
# matplotlib

from ipywidgets import interactive,widgets
from IPython.display import display,Math,HTML
from matplotlib import pyplot
from sympy import isprime, mod_inverse
import ipysheet
from ipysheet import calculation
import math
import decimal
import pandas

# Theoretische Grundlagen

## Zuordnung kryptografische Mechanismen zu Schutzzielen

Mechnismus       | Beispiele                       | Vertraulichkeit | Integritaet | Authentizitaet | Verbindlichkeit | Verfuegbarkeit 
----------------:|--------------------------------:|:---------------:|:-----------:|:--------------:|:---------------:|:--------------:
Hash-Funktion    |MD5,SHA-1, SHA256                |                 | X           |                |                 |
MAC              |HMAC                             |                 | X           | X              |                 |
Verschluesselung |RSA,ElGamal,Transpositionschiffre| X               |             |                |                 |
Signaturen       |RSA,ElGamal                      |                 | X           | X              | X               |

## Definition Einwegfunktion

Eine Funktion $y = f(x)$ heisst Einwegfunktion wenn gilt:
  - Es gibt ein effizientes Verfahren zur Berechnung von $y=f(x)$.
  - Es gibt kein effizientes Verfahren zur Berechnung von $x=f^{-1}(y)$.

## Definition Trapdoor Einwegfunktionen

- Eine Variante der Einwegfunktion, welche effizient invertierbar ist, wenn man gewisse Zusatzinformationen besitzt.

## Symmetrische Verfahren der Verschluesselung

- substitutionsalgorithmen
    - monoalphabetische chiffre
      - additive chiffre

        verschluesselung: $e_K(x) = x + K \mod n$

        entschluesselung: $d_K(y) = y - K \mod n$

      - multiplikative chiffre

        verschluesselung: $e_K(x) = ax \mod n$

        entschluesselung: $d_K(y) = a^{-1}y \mod n$

        __Obacht:__ $a$ muss teilerfremd zu $n$ sein, da fuer die entschluesselung $a^{-1}$ benoetigt wird.

    - polyalphabetische chiffre

      => Die haefigkeit der Buchstaben wird verschleiert

      __Obacht:__ die Haeufigkeit von Buchstabengruppen wird nicht verschleiert

    - one-time-pad

      => Perfekt, wenn key wirklich zufaellig und gleich gross wie klartext ist

- Transpositionschiffren

  => Die position der zeichen wird getauscht
  

## Asymetrische Verfahren

### RSA

#### Mathematische Grundlage
- Basiert auf dem Faktorisierungsproblem

#### Angrifsmoeglichkeiten
  - Public-Key-Only-Attacke
  - Chosen-Ciphertext-Attacke
  - Chosen-Plaintext angriffe
  
  => Der private Schluessel kann aus dem oeffentlichen berechnet werden, wenn $\phi(n)$ oder $n=p \cdot q$ aus $n$ ermittelt werden koennen.
  
  - Bruteforce
     - Nur bei kurzen Schluessellaengen moeglich, weil extrem aufwendig. 
     
  - MITM 
     - Falls keine Signatur verwendet wird kann der Angreifer sich zu beginn der Kommunikation als eigentlicher Kommunikationspartner ausgeben.

### ElGamal

#### Mathematische Grundlage
- Basiert auf dem diskreten Logarithmus Problem

# Mathematische Grundlagen

## Gruppen auf endlichen Mengen

### Additive Gruppe

#### Definition

$\underline{G} = \{0, 1, 2, 3, ..., m-1\}$, wobei $m > 0$

$a \oplus b = r$

$\frac{a+b}{m}=q+\frac{r}{m} $$\iff$$ a + b = q \cdot m + r $


#### Addition

In [2]:
def gadd(m,a,b):
    out = (a+b)%m
    display(Math(str(a)+"\oplus" +str(b)+"="+str(out)))
    return out

w = interactive(
    gadd,
    m=widgets.IntText(value=5, description="m:"),
    a=widgets.IntText(value=2, description="a:"),
    b=widgets.IntText(value=4, description="b:"),
)

display(w)

interactive(children=(IntText(value=5, description='m:'), IntText(value=2, description='a:'), IntText(value=4,…

#### Additiv inverses Element

$a \oplus (-a) = a \oplus (m - a) = 0$
$\iff$
$\frac{a+(m-a)}{m}=1+\frac{0}{m}$

### Multiplikative Gruppe

#### Definition

$\underline{G} = \{0, 1, 2, 3, ..., p-1\}$, wobei $p$ eine Primzahl ist.

$a \odot b = r$

$\frac{a \cdot b}{p}=q+\frac{r}{p} $$\iff$$ a \cdot b = q \cdot p + r $


#### Multiplikation

In [3]:
def gmul(p,a,b):
    out = (a*b)%p
    display(Math(str(a)+'\cdot'+str(b)+'='+str(out)))
    return out

w = interactive(
    gmul,
    p=widgets.IntText(value=5, description="p:"),
    a=widgets.IntText(value=2, description="a:"),
    b=widgets.IntText(value=2, description="b:"),
)

display(w)

interactive(children=(IntText(value=5, description='p:'), IntText(value=2, description='a:'), IntText(value=2,…

#### Multiplikativ inverses Element

$ c \odot a = a \odot c = 1 $$\iff$$ c \cdot a = q \cdot p + 1$

In [4]:
table = []

def mulInv(n, a, depth =  1):
    table.append({"n": n, "a": a, "q": int(n/a), "r": n%a})
    if n%a == 0:
        return depth
    return mulInv(a, n%a, depth+1)
                  
def calcXy():
    first = True
    lastY = 1
    for i in reversed(table):
        if first:
            i['x'] = 0
            i['y'] = 1
            first = False
        else:
            i['x'] = lastY
            i['y'] = (1-i['n']*i['x'])/i['a']
            lastY = i['y']

def getAInverse():
    row = table[0]

    display(Math(str(row['n']) + '\cdot' + str(row['x']) + '+' + str(row['a']) + '\cdot' + str(row['y']) + '=1 \mod ' + str(row['n'])))
    display(Math('a \cdot a^{-1} = 1'))
    display(Math(str(row['a']) + '\cdot a^{-1} = 1'))
    display(Math(str(row['a']) + '\cdot' + str(row['y']) + '=1'))
    
    if (row['y'] > 0):
        return row['y']
    else:
        return row['n']+row['y']
            
def printTable():
    d = pandas.DataFrame.from_dict(table)
    display(d)

def doFun(n,a):
    table.clear()
    mulInv(n,a)
    calcXy()
    display(Math('nx+ay=1'))
    display(Math('y=\\frac{1-nx}{a}'))
    printTable()
    display(Math('a^{-1} =' + str(getAInverse())))
    
    
w = interactive(
    doFun,
    n=widgets.IntText(value=5, description="n:"),
    a=widgets.IntText(value=2, description="a:"),
)

display(w)

interactive(children=(IntText(value=5, description='n:'), IntText(value=2, description='a:'), Output()), _dom_…

### Definition Zahlenkoerper

Die Menge $\underline{F}$ sei ein Zahlenkoerper bezueglich der Verknuepfungen $\oplus$ und $\odot$, wenn folgende Bedingungen erfuellt sind. 

  1. $\underline{F}$ ist bezueglich $\oplus$ eine additive Gruppe. Das Identitaetselement dieser Gruppe sei das Nullelement.
  2. Die von Null verschiedenen Elemente von $\underline{F}$  bilden eine multiplikative Gruppe bezueglich $\odot$. Das Identitaetselement dieser Gruppe sei das Einselement.
  3. Die Multiplication $\odot$ ist distributiv bezueglich der Addition $\oplus$, so dass gilt $a \odot (b \oplus c) = (a \odot b)\oplus(a \odot c)$ fuer $a, b, c \in \underline{F}$

### Modulare reduktion

#### Beispiel

$g^{a+b} = g^b \cdot g^b$

$17^{23} \mod 26$

aufsplitten funktioniert nur wenn $g^a > p$ 

$17^{23} = 17^4 \cdot 17^4 \cdot 17^4 \cdot 17^4 \cdot 17^4 \cdot 17^4 \cdot 17^1 = $

$17^4 \mod 26 = 9$

$17^{23} = 9 \cdot 9 \cdot 9 \cdot 9 \cdot 9 \cdot 9 \cdot 17 = 9^3 \cdot 9^3 \cdot 17$

$9^3 \mod 26 = 1$

$17^{23} = 1 \cdot 1 \cdot 17$

#### Code

In [5]:
def modRed(g, a, p, reduceTo):
    display(Math(str(g) +'^{' + str(a) + '} \mod ' + str(p)))
    display(Math('\iff'))
    
    np = int(a / reduceTo)
    nl = a % reduceTo

    string = ''
    for i in range(np):
        string += str(g) + '^' + str(reduceTo) + ' \mod ' + str(p) + ' \cdot '
    string += str(g) + '^{' + str(nl) + '} \mod ' + str(p)
    display(Math(string))
    
    reduced = (g**reduceTo)%p
    display(Math(str(g) + '^' + str(reduceTo) + ' \mod ' + str(p) + ' = ' + str(reduced))) 
    reduced2 = (g**nl)%p
    display(Math(str(g) + '^{' + str(nl) + '} \mod ' + str(p) + ' = ' + str(reduced2)))
    
    result = (reduced**np*reduced2)%p
    display(Math(str(reduced) + '^{' + str(np) + '} \cdot ' + str(reduced2) + ' \mod ' + str(p) + ' = ' + str(result)))
          
w = interactive(
    modRed,
    g=widgets.IntText(value=5, description="a:"),
    a=widgets.IntText(value=2, description="a:"),
    p=widgets.IntText(value=5, description="p:"),
    reduceTo=widgets.IntText(value=2, description="recude to:"),
)  

display(w)

interactive(children=(IntText(value=5, description='a:'), IntText(value=2, description='a:'), IntText(value=5,…

### Eulersche $\phi$-Funktion

#### Satz von Euler
Fuer eine gegebene natuerliche Zahl $n$ gilt:

$a^{\phi(n)} = a^0 = 1 \mod n$ fuer $a>0$ und $ggT(n,a)=1$

Der Satz gilt fuer alle positiven und ganzzahligen Werte von $a$, die teilerfremd zu $n$ sind.

- Der Satz von Euler kann genutzt werden um das multiplikativ inverse Element $a^{-1}$ zu berechnen.

=> auf dem Exponenten kann modulo $\phi(n)$ gerechnet werden:

$a^{j \cdot \phi(n)) \mod \phi(n)} = a^0 = 1 \mod n$

Unter der Annahme, dass $n$ eine Primzahl ist und $\phi(n) = n - 1$ gilt, muss gelten:

$a^{n-1} = 1 \mod n$ fuer $a>0$ und $ggT(n,a)=1$



## GGT berechnen

In [6]:
def ggt_v01(x, y):
    while x > 0 and y > 0:
        if x >= y:
            x = x - y
        else:
            y = y - x
    return x+y

def ggt_v02(x, y):
    rest = 1
    while rest != 0: 
        if x >= y:
            rest = x % y
            x = y
            y = rest
            ggt = x
        else:
            rest = y % x
            y = x
            x = rest
            ggt=y
    return ggt

def ggt(x,y):
    display("GGT Ansatz 1: "+str(ggt_v01(x,y)))
    display("GGT Ansatz 2: "+str(ggt_v02(x,y)))
    
w = interactive(
    ggt,
    x=widgets.IntText(value=5, description="p:"),
    y=widgets.IntText(value=2, description="q:"),
)  

display(w)

interactive(children=(IntText(value=5, description='p:'), IntText(value=2, description='q:'), Output()), _dom_…

# Symetrische Kryptographie

## Caesar

### Verschluesselung

In [7]:
def caesar_encrypt(key, text):
    verschluesselter_text=""
    for zeichen in text.upper():
        zahl = ord(zeichen)
        neue_zahl = zahl + key
        if neue_zahl > ord('Z'):
            neue_zahl = neue_zahl - 26
        neuesZeichen = chr(neue_zahl)
        verschluesselter_text = verschluesselter_text + neuesZeichen
    display(verschluesselter_text)
    return verschluesselter_text

w = interactive(
    caesar_encrypt,
    key=widgets.IntText(value=5, description="Schluessel:"),
    text=widgets.Text(value="Klartext", description="Klartext:"),
)  

display(w)

interactive(children=(IntText(value=5, description='Schluessel:'), Text(value='Klartext', description='Klartex…

### Entschluesselung

In [8]:
def caesar_decrypt(key, text):
    decrypted = ""
    for zeichen in text.upper():
        zahl = ord(zeichen)
        neue_zahl = zahl - key
        if neue_zahl < ord('A'):
            neue_zahl = neue_zahl + 26
        if neue_zahl == 58:
            neue_zahl = 32
        neuesZeichen = chr(neue_zahl)
        decrypted = decrypted + neuesZeichen
    display(decrypted)
    return decrypted

w = interactive(
    caesar_decrypt,
    key=widgets.IntText(value=5, description="Schluessel:"),
    text=widgets.Text(value="Geheimtext", description="Geheimtext:"),
)  

display(w)

interactive(children=(IntText(value=5, description='Schluessel:'), Text(value='Geheimtext', description='Gehei…

## Transpositionschiffre

### Spaltentransposition

In [9]:
def spaltentransposition(inp,spalten):
    rows = math.ceil(len(inp)/spalten)
    Matrix = []
    Matrix = [[" " for x in range(spalten)] for y in range(rows)] 

    c = 0
    for char in inp:
        column = math.floor(c / rows)
        row = math.floor(c% rows+1)-1
        Matrix[row][column] = char
        c += 1

    d = pandas.DataFrame.from_dict(Matrix)
    display(d)
    
w = interactive(
    spaltentransposition,
    spalten=widgets.IntText(value=5, description="Spalten:"),
    inp=widgets.Text(value="Text", description="Text:"),
)  

display(w)

interactive(children=(Text(value='Text', description='Text:'), IntText(value=5, description='Spalten:'), Outpu…

## Angriffe

### Haeufigkeitsanalse

import numpy as np
import matplotlib.pyplot as plt


filename = input ("Filename bitte:")       # unsere Files Klartext und encrypted aus Wireshark
infile   = open(filename, "r")
s0       = infile.read()                   # die Datei ist in der Zeichenkette s0 gespeichert
s        = s0.lower()                      # alle die Grossbuchstaben werden klein geschrieben
infile.close()

a_ASCII = ord('a')
absolute_haeufigkeit = [0]*26
anz_buchstaben = 0

for x in s:
    if x.isalpha():                 # x ist ein Charakter
        absolute_haeufigkeit[ord(x) - a_ASCII] += 1
        anz_buchstaben                         += 1

graphx =[]
for i in range(26):
    graphx.append(chr(a_ASCII + i))
    if absolute_haeufigkeit[i] == 0:
        print(chr(a_ASCII + i) + " taucht nicht auf!\t\t" + 
              "{0: >6.3f}".format(absolute_haeufigkeit[i]/anz_buchstaben) + "%")
    elif absolute_haeufigkeit[i] == 1:
        print(chr(a_ASCII + i) + " taucht 1 Mal auf:\t" + 
              "{0: >6.3f}".format(absolute_haeufigkeit[i]/anz_buchstaben) + "%")
    else:
        print(chr(a_ASCII + i) + " taucht " + str(absolute_haeufigkeit[i]) + " Mal:\t" + 
              "{0: >6.3f}".format(100*absolute_haeufigkeit[i]/anz_buchstaben) + "%")

print("\nGesamtzahl der Buchstaben:", anz_buchstaben)

print ("\nBuchstabe", chr(a_ASCII + np.argmax(absolute_haeufigkeit)), "mit einer Häufigkeit von",np.max(absolute_haeufigkeit))

plt.bar(graphx,absolute_haeufigkeit)
plt.show()

# Asymetrische Kryptographie

## Diffie-Hellman Schlüsselaustausch

### Definition

Alice und Bob vereinbaren Modul $p$ und Basis $g$

Alice:
 - Erstellt (geheim): $a$
 - Berechnet: $\alpha = g^a \mod p$
 - Sendet $\alpha$ zu Bob
 - Empängt $\beta$ von Bob
 - Berechnet: $\beta^a \mod p$

Bob:
 - Erstellt (geheim): $b$
 - Berechnet: $\beta = g^b \mod p$
 - Sendet $\beta$ zu Alice
 - Empängt $\alpha$ von Alice
 - Berechnet: $\alpha^b \mod p$
 
 $K = g^{a \cdot b} \mod p = \beta^a \mod p = \alpha^b \mod p$

### Code

In [10]:
def calc(g,a,p):
    return (g**a)%p

def dh(p,g,a,b):
    alpha = calc(g,a,p)
    display(Math('\\alpha = ' + str(g) + '^{' + str(a) + '} \mod ' + str(p) + ' = ' + str(alpha)))
    beta = calc(g,b,p)
    display(Math('\\beta = ' + str(g) + '^{' + str(b) + '} \mod ' + str(p) + ' = ' + str(beta)))
    alpha_b = calc(alpha,b,p)
    display(Math('K_1 = ' + str(alpha) + '^{' + str(b) + '} \mod ' + str(p) + ' = ' + str(alpha_b)))
    beta_a = calc(beta,a,p)
    display(Math('K_2 = ' + str(beta) + '^{' + str(a) + '} \mod ' + str(p) + ' = ' + str(beta_a)))

    if (alpha_b == beta_a):
        display(Math('K= ' + str(beta_a) ))
        
w = interactive(
    dh,
    p=widgets.IntText(value=1, description="p:"),
    g=widgets.IntText(value=2, description="g:"),
    a=widgets.IntText(value=3, description="a:"),
    b=widgets.IntText(value=4, description="b:"),
)  

display(w)

interactive(children=(IntText(value=1, description='p:'), IntText(value=2, description='g:'), IntText(value=3,…

## ElGamal Schluesselvereinbarung

### Definition

Alice:
  - bekommt $K_{pub} = (p,g,e)$ von Bob
  - ueberprueft ob  $K_{pub}$ wirklich zu Bob gehoert
  - waehlt Zufallszahl $k$
  - Berechnet Sitzungsschluessel $K=e^k \mod p$
  - Berechnet Schluesselwert $a=g^k \mod p$
  - Sendet $a$ an Bob
    
Bob:
  - $K_{pr} = d$
  - Empfaengt $a$ von Alice
  - Berechnet $K = a^d \mod p$
    
Verfikation:
    $a^d = g^{d \cdot k} = e^k = K \mod p$

### Schwachstellen

- Bob hat keine Sicherheit ueber die Herkunft von $K$, da jeder seinen $K_{pub}$ nutzen kann und $a$ kein Merkmal von Alice enthaelt

### Code

#### Alice

In [11]:
def calc(a,b,c):
    return (a**b)%c

def alice(p,g,e,k):
    K = calc(e,k,p)
    a = calc(g,k,p)
    display(Math('K = ' + str(e) + '^{' + str(k) + '} \mod ' + str(p) + ' = ' + str(K)))
    display(Math('a = ' + str(g) + '^{' + str(k) + '} \mod ' + str(p) + ' = ' + str(a)))
    
w = interactive(
    alice,
    p=widgets.IntText(value=1, description="p:"),
    g=widgets.IntText(value=2, description="g:"),
    e=widgets.IntText(value=3, description="e:"),
    k=widgets.IntText(value=4, description="k:"),
)  

display(w)

interactive(children=(IntText(value=1, description='p:'), IntText(value=2, description='g:'), IntText(value=3,…

#### Bob

In [12]:
def calc(a,b,c):
    return (a**b)%c

def bob(d,p,a):
    K = calc(a,d,p)
    display(Math('K = ' + str(a) + '^{' + str(d) + '} \mod ' + str(p) + ' = ' + str(K)))
    
w = interactive(
    bob,
    d=widgets.IntText(value=1, description="d:"),
    p=widgets.IntText(value=2, description="p:"),
    a=widgets.IntText(value=3, description="a:"),
)  

display(w)

interactive(children=(IntText(value=1, description='d:'), IntText(value=2, description='p:'), IntText(value=3,…

## RSA

### Definition

$K_{pub}=(n,e)$

$K_{pr}=(d)$

Eine Verschluesselungsfunktion $e()$ muss umkehrbar sein => es muss eine Funktion $d()$ geben, welche die Wirkung von $e()$  rueckgaenig macht:

$d_K(y) = d_K(e_K(x)) = x$

Die Kombination von Ver- und Entschluesselung fuehrt dabei auf 

$d_K(y) = d_K(e_K(x)) = e^{e \cdot d} = x^1 = x \mod n$

Mit der angegebenen Bedingung in der Form

$ d \cdot e = 1 \mod \phi(n)$

kann aus dem Geheimtext wieder der Klartet zurueckgewonnen werden.

Wird der Satz von Euler mit $i$ potenziert und mit $a$ multipliziert ergibt sich:

$a^{i \phi(n)+1} = a^{i \phi(n))} \cdot a = 1 \cdot a = a \mod n$

Die wesentliche Gleichung fuer RSA ergibt sich, indem der Exponent 

$i \cdot \phi(n) + 1$ druch $ e \cdot d$ ersetzt wird ($\mod (\phi(n))$)

Wenn $n$ eine Primzahl ist kann die Eulersche Phi-Funktion berechnet werden und die berechnung des geeheimen Schluessels ist moeglich:

$d \cdot e = 1 \mod \phi(n)$

### Key erzeugen

In [13]:
def rsa_key(p,q,e=6157):
    n = p*q
    phi_n = (p-1)*(q-1)
    counter = 0
    while math.gcd(e,phi_n)>1:
        counter+=1
        e = e+1
        if counter > 1000000:
            display("Endlosschleife bei gcd(e,phi_n)")
            return (0,0,0)
    d = 0
    h = 2
    counter = 0
    while h!=1:
          d = d + 1
          h = e * d % phi_n
          if counter > 1000000:
              display("Endlosschleife bei h!=1")
              return (0,0,0)
    return (n,d,phi_n)

def doRsa(p,q,e):
    if (not isprime(p)):
        display("p ist keine Primzahl! -> Private Key kann berechnet werden")
    if (not isprime(q)):
        display("q ist keine Primzahl! -> Private Key kann berechnet werden")
    n,d,phi_n = rsa_key(p,q,e)
    if (e <= 1 or e >= phi_n):
        display("e ist schlecht gewaehlt. fuer e sollte folgendes gelten: 1 < e < phi(n)")
    display(Math('n = ' + str(n)))
    display(Math('\phi(n) = ' +  str(phi_n)))
    display(Math('K_{pub} = ('+  str(n) + ',' + str(e) +')'))
    display(Math('K_{pr} = ('+  str(d) +')'))

w = interactive(
    doRsa,
    p=widgets.IntText(value=1, description="p:"),
    q=widgets.IntText(value=2, description="q:"),
    e=widgets.IntText(value=6157, description="e:"),
)  

display(w)

interactive(children=(IntText(value=1, description='p:'), IntText(value=2, description='q:'), IntText(value=61…

### Verschluesseln

$ y=e_{K_{pub}}(x)=x^e \mod n $

In [14]:
def rsa_encrypt(n,e,x):
    y =x**e % n
    display("y (Geheimtext): "+str(y))
    return y 

w = interactive(
    rsa_encrypt,
    n=widgets.IntText(value=1, description="n (Public):"),
    e=widgets.IntText(value=2, description="e (Public):"),
    x=widgets.IntText(value=6157, description="x (Klartext):"),
)  

display(w)

interactive(children=(IntText(value=1, description='n (Public):'), IntText(value=2, description='e (Public):')…

### Entschluesseln

$ x=d_{K_{pr}}(y)=y^d \mod n $

In [15]:
def rsa_decrypt(n,d,y):
    x =y**d % n
    display("x (Klartext): "+str(x))
    return x 

w = interactive(
    rsa_decrypt,
    n=widgets.IntText(value=1, description="n (Public):"),
    d=widgets.IntText(value=2, description="d (Private):"),
    y=widgets.IntText(value=6157, description="y (Geheimtext):"),
)  

display(w)

interactive(children=(IntText(value=1, description='n (Public):'), IntText(value=2, description='d (Private):'…

### Signieren

$K_{pr} = d$

$s=e_{K_{pr}}(x) = x^d \mod n$

=> Es wird $s$ und $x$ veroeffentlicht. Die signatur kann von jedem mit $K_{pub}$ geprueft werden

In [16]:
def sign(d,n,x):
    s = x**d%n
    display(Math('s=x^d \mod n'))
    display(Math('s=' + str(x) + '^{' + str(d) +'} \mod ' + str(n)  + '=' + str(s)))
    
w = interactive(
    sign,
    d=widgets.IntText(value=1, description="d (Private):"),
    n=widgets.IntText(value=2, description="n (Public):"),
    x=widgets.IntText(value=6157, description="x (Nachricht):"),
)  

display(w)

interactive(children=(IntText(value=1, description='d (Private):'), IntText(value=2, description='n (Public):'…

### Signatur ueberpruefen

$K_{pub} = (n, e)$

$x = d_{K_{pub}}(s) = s^e \mod n$

=> Wenn das berechnete $x$ mit der Nachricht $x$ uebereinstimmt wurde die Nachricht nicht veraendert

In [17]:
def checkSignature(n,e,s):
    x = s**e%n
    display(Math('x=s^e \mod n'))
    display(Math('x=' + str(s) + '^{' + str(e) +'} \mod ' + str(n)  + '=' + str(x)))
    
w = interactive(
    checkSignature,
    n=widgets.IntText(value=1, description="n (Public):"),
    e=widgets.IntText(value=2, description="e (Public):"),
    s=widgets.IntText(value=6157, description="s (Signatur):"),
)  

display(w)

interactive(children=(IntText(value=1, description='n (Public):'), IntText(value=2, description='e (Public):')…

## ElGamal

Nachteil bzgl. RSA:

=> Höherer Speicheraufwand

### Definition

Oeffentlich bekannt:
 - Eine grosse Primzahl $p$
 - eine Basis $g$ aus dem Galois-Koerper $GF(p)$
    
Jeder Teilnehmer waehlt jeweils einen privaten Schluessel
$K_{pr} = d$
und berechnet den zugehoerigen oeffentlichen Schluessel $K_{pub} = e$ 
ueber
$e=g^d \mod p$

=> Die Ermittlung des privaten Schluessels $K_{pr} =d$ aus dem oeffentlichen  ist bei genuegend grossem Modul $p$ nicht durchfuehrbar.

### Verschluesselung

$K_{pub} = (p,g,e)$

$a=g^k \mod p$

$b=e^k \cdot m \mod p$

der Geheimtext besteht aus $a$ und $b$

k muss teilerfremd zu p-1 sein. 

__Obacht__: Aus $g^k$ muss theoretisch jedes beliebige $a$ erzeugt werden. Falls $g^k$ nicht primitiv ist, wären die möglichen element von a schon von vornherein eingeschränkt. 

In [18]:
def elGamal_encrypt(p,g,e,k,m):
    if (math.gcd(k,p-1) != 1):
        display("k ist nicht teilerfremd zu p-1 -> kann gebrochen werden")
    a = g**k%p
    b = ((e**k)*m)%p
    display(Math('a=g^k \mod p'))
    display(Math('a=' + str(g) + '^{' + str(k) +'} \mod ' + str(p)  + '=' + str(a)))
    display(Math('b=e^k \cdot m \mod p'))
    display(Math('b=' + str(e) + '^{' + str(k) +'} \cdot ' + str(m) + ' \mod ' + str(p)  + '=' + str(b)))
    
w = interactive(
    elGamal_encrypt,
    p=widgets.IntText(value=1, description="p (Public):"),
    g=widgets.IntText(value=2, description="g (Public):"),
    e=widgets.IntText(value=6157, description="e (Public):"),
    k=widgets.IntText(value=2, description="k (Zufall):"),
    m=widgets.IntText(value=2, description="m (Klartext):"),
)  

display(w)   

interactive(children=(IntText(value=1, description='p (Public):'), IntText(value=2, description='g (Public):')…

### Entschluesselung

$K_{pr} = d$

$m = a^{p-d-1} \cdot b \mod p$

In [19]:
def elGamal_decrypt(d,p,a,b):
    m = (a**(p-d-1))*b%p
    display(Math('m = a^{p-d-1} \cdot b \mod p'))
    display(Math('m=' + str(a) + '^{' + str(p) + '-' + str(d) + '-1} \cdot ' + str(b) + '\mod ' + str(p)  + '=' + str(m)))
    
w = interactive(
    elGamal_decrypt,
    d=widgets.IntText(value=1, description="d (Private):"),
    p=widgets.IntText(value=2, description="p (Public):"),
    a=widgets.IntText(value=6157, description="a (Geheimtext):"),
    b=widgets.IntText(value=2, description="b (Geheimtext):"),
)  

display(w)  

interactive(children=(IntText(value=1, description='d (Private):'), IntText(value=2, description='p (Public):'…

### Signieren

In [45]:
def elGamal_sign(m,d,p,g,r):
    # todo: pruefen ob r teilerfremd zu p-1 ist
    r1 = mod_inverse(r,(p-1))
    display(Math("r\\cdot r^{-1}=1\mod (p-1) => r =" + str(r) + " => r^{-1} = " + str(r1)))
    rho = (g**r)%p
    display(Math('\\rho=g^r\mod p = ' + str(g) + "^" + str(r) + "\mod" + str(p) + "=" + str(rho)))
    s = ((m-d*rho)*r1)%(p-1)
    display(Math('s=(m-d\cdot \\rho)\cdot r^{-1} \mod (p-1) = (' + str(m) + "-" + str(d) + "\cdot" + str(rho) + ') \cdot' + str(r1) + "\mod (" + str(p) + "-1) =" + str(s)))
    display(Math("(m,\\rho,s)=(" + str(m) + "," + str(rho) + "," + str(s) + ")"))
    
w = interactive(
    elGamal_sign,
    m=widgets.IntText(value=8, description="m (Nachricht):"),
    d=widgets.IntText(value=3, description="d (Private):"),
    p=widgets.IntText(value=13, description="p (Public):"),
    g=widgets.IntText(value=7, description="g (Public):"),
    r=widgets.IntText(value=7, description="r (Zufallszahl):"),
)  

display(w)  

interactive(children=(IntText(value=8, description='m (Nachricht):'), IntText(value=3, description='d (Private…

### Signatur verifizieren

In [43]:
def elGamal_verify(m,rho,s,p,g,e):
    display(Math("g^m \equiv e^\\rho\cdot\\rho^s\mod p"))
    l = g**m%p
    display(Math("g^m \mod p="+str(g)+"^"+str(m)+"\mod " + str(p) + "="+str(l)))
    r = e**rho*rho**s%p
    display(Math("e^\\rho\cdot\\rho^s\mod p="+str(e)+"^"+str(rho)+"\cdot"+str(rho)+"^"+str(s)+"\mod " + str(p) + "="+str(r)))
    if (l==r):
        display("Signatur gueltig!")
    else:
        display("Signatur ungueltig!")
    
w = interactive(
    elGamal_verify,
    m=widgets.IntText(value=8, description="m (Nachricht):"),
    rho=widgets.IntText(value=6, description="$\\rho$ (Signatur):"),
    s=widgets.IntText(value=2, description="s (Signatur):"),
    p=widgets.IntText(value=13, description="p (Public):"),
    g=widgets.IntText(value=7, description="g (Public):"),
    e=widgets.IntText(value=5, description="e (Public):"),
)  

display(w)  
    

interactive(children=(IntText(value=8, description='m (Nachricht):'), IntText(value=6, description='$\\rho$ (S…

# Elliptische Kurven

## $y^2 = x^3 + 3x$

In [20]:
def gradient(px, py, qx, qy, mod):
    a = (py-qy)%mod
    b = (px-qx)%mod
    b2 = mod_inverse(b,mod)
    m = a*b2
    display("1.) Anstieg der Geraden")
    display(Math("m=\\frac{y_p-y_q}{x_p-x_q}=\\frac{"+ str(py) +"-" + str(qy) + "}{" + str(px) + "-" + str(qx) + "} \mod "+ str(mod) + "= \\frac{"+str((py-qy)%mod) +"}{"+str((px-qx)%mod) + "} \mod" + str(mod) + "=" + str(a) + "\cdot" + str(b) + "^{-1} \mod" + str(mod) + "=" + str(m)))
    return m
    
def calc_xr(m, px, qx, mod):
    xr = (m*m-px-qx)%mod
    display("2.) Xr-Koordinate des Punktes (-R) bzw. (R)")
    display(Math("x_r=m^2-x_p-x_q=" + str(m)+"^2-" + str(px) + "-" + str(qx) + "\mod " + str(mod) + "=" + str(xr)))
    return xr

def calc_yr(m, py, px, xrp, mod):
    yrp = (py-m*(px-xrp))%mod
    display("3.) Yr-Koordinate des Punktes (-R)")
    display(Math("y_r=y_p-m\cdot(x_p-x_r)=" + str(py)+"-" + str(m) + "\cdot(" + str(px) + "-" + str(xrp) + ")\mod " + str(mod) + "=" + str(yrp)))
    return yrp

def negotiation(xr,yr,mod):
    yrp = -yr%mod
    display("4.) Punktnegation")
    display(Math("P+Q=R=(x_r|-y_r)=(" + str(xr) + "|-" + str(yr) +")=(" + str(xr) + "|" + str(yrp) + ")"))
    return yrp

def verification(xr, yrp, yrm, mod):
    xc = (xr*xr*xr+3*xr)%mod
    y2p = (yrp*yrp)%mod
    y2m = (yrm*yrm)%mod
    
    if (y2p != xc or y2m != xc):
        display("Fuck")
    
    display("5.) Verifikation")
    display(Math("y^2=x^3+3x=" + str(yrp) + "^2=" + str(y2p) + "=" + str(yrm) + "^2=" + str(y2m) + "=" + str(xr) + "^3+3\cdot" + str(xr) +"=" +str(xc)))
    
def intersection(px,py,qx,qy,mod):
    m = gradient(px,py,qx,qy,mod)
    xr = calc_xr(m, px, qx, mod)
    yrm = calc_yr(m, py, px, xr, mod)
    yrp = negotiation(xr,yrm,mod)
    verification(xr,yrp, yrm, mod)

w = interactive(
    intersection,
    px=widgets.IntText(value=2, description="$P_x$"),
    py=widgets.IntText(value=2, description="$P_y$"),
    qx=widgets.IntText(value=4, description="$Q_x$"),
    qy=widgets.IntText(value=1, description="$Q_y$"),
    mod=widgets.IntText(value=5, description="GF"),
)  

display(w)

interactive(children=(IntText(value=2, description='$P_x$'), IntText(value=2, description='$P_y$'), IntText(va…